# 1. 준비
- cpu 환경은 너무 느려서 코랩에서 부탁드립니다 ㅎ
- 꼭 GPU 가속이 켜져있는지 확인해주세요!
- 추가로, 구글 드라이브에 학습 데이터, 사전 데이터, 모델을 모두 로드해 주세요!

In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-eofaoxhr
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-eofaoxhr
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00

In [24]:
! pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602642 sha256=ac042ffdb003fdf969c1d2497546d7bd41f3716d724f783d19c8dcf4bf8234c2
  Stored in directory: /root/.cache/pip/wheels/9e/34/31/a1682c8249bf2ba89a29498d62c35c08ceb116537b4530d93e
Successfully built kiwipiepy-model


In [3]:
# 드라이브 위치를 알맞게 조정하세요!
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/MyDrive/종합설계 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/종합설계


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/종합설계/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/종합설계/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# 2. 데이터 준비
- 여기서는 training_dataset.csv를 활용해서 train, test set을 준비하게 됩니다!
- 제가 디렉토리에 같이 넣어놀테니까 이거 활용하세요! --> 구글 드라이브로 옮기기
- 추가로, 사전도 미리 준비해 놓겠습니다!

In [5]:
import pandas as pd
data = pd.read_csv("training_dataset.csv")

data_list = list()
for sen, lab in zip(data["0"], data["1"]):
  data_list.append([sen,lab])

In [7]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data_list, test_size=0.1, random_state=0) # train : test = 9:1
train_set[0]

# 여기서 만들어진 test_set을 중점으로 활용하시면 됩니다!

['그 동안 보빨러들이 얼마나 잘 해줬겠어ㅋㅋㅋ', 2]

In [9]:
test_set[:3]

# 아래를 보면 알겠지만, 각 row마다 0번이 문장 데이터, 1번이 라벨링 데이터입니다!

[['인정 쟤네 목소리 쫌 심각하게 족같아', 1],
 ['나 알바하는 곳 너무 장사 잘돼서 힘들어.', 0],
 ['아슬아슬~ 정말 프로답게 벗어 주네.', 2]]

In [10]:
abuse = pd.read_csv("폭언사전.csv")
sexual = pd.read_csv("성희롱사전.csv")

# 사전은 아래에서 쓰일 거라서 미리 로드합니다!

In [11]:
abuse.head(3)

,Unnamed: 0,0
0,0,가난
1,1,가난뱅이
2,2,가두


In [12]:
sexual.head(3)

# 사전이 dataframe으로 로드되면서, 컬럼명[Unnamed:0, 0]으로 자동 지정되었어요!
# 여기서 '0' col을 사용하면 되겠죠?

,Unnamed: 0,0
0,0,19금
1,1,69
2,2,69자세


In [13]:
abuse_list = list()
sexual_list = list()

for rows in abuse["0"]:
  abuse_list.append(rows)

for rows in sexual["0"]:
  sexual_list.append(rows)

# dataframe로 load한 사전이 사용하기 불편할 수도 있으니, 리스트 형식으로도 저장합니다

In [14]:
abuse_list[:3], sexual_list[:3]

(['가난', '가난뱅이', '가두'], ['19금', '69', '69자세'])

# 3. 코버트 모델 로드
- 여기서는 저희가 만든 "kobert_classifier.pth"을 로드해서 모델을 준비합니다
- 모델은 가지고 계시죠? 그거 구글 드라이브에 로드해서 사용하시면 됩니다!

In [15]:
device = torch.device("cuda:0")

class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):

    # sentence , label data를 BERT의 입력값에 맞게 변환하는 transformer를 생성
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_len, pad=pad, pair=pair)

    ## 생성한 transformer로 sentence를 변환하여 저장
    self.sentences = [transform([data[sent_idx]]) for data in dataset]
    self.labels = [np.int32(data[label_idx]) for data in dataset]
  
  def __getitem__ (self, i):
    return (self.sentences[i] + (self.labels[i], )) # 각 index에 맞는 item 반환 진행 --> 왜 이런 형태인지는 잘 모르겠음
  
  def __len__(self):
    return (len(self.labels))

# Parameter setting 진행
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

# Kobert 모듈에서 제공하는 get_tokenizer와 vocab를 활용해 tokneizer를 구성한다
tokenizer = get_tokenizer() 
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_set, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_set, 0, 1, tok, max_len, True, False)

class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size = 768, num_classes=3, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    ## classifier는 선형 회귀 모델로 구성 (input size = 768, output size = 3 (label이 3개로 구성))
    self.classifier = nn.Linear(hidden_size, num_classes)

    ## overfitting 방지를 위한 dropout 비율 설정
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  # attention mask sequence를 구성해주는 함수 --> padding이 아닌 영역을 0에서 1로 변경
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i,v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    
    return attention_mask.float()
  
  # bert + classifier를 관통하는 forward 연산 진행
  def forward(self, token_ids, valid_length, segment_ids):

    # attention_mask 계산
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    # bert에 input 투입, 변수명이 pooler인거 보니 출력 embedding에 mean pooling 적용한 값이지 않을까 추측
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))

    # dropout 비율이 존재한다면, dropout 적용
    if self.dr_rate:
        out = self.dropout(pooler)

    # classifier 진행
    return self.classifier(out) 

model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model_state_dict = torch.load("kobert_classifier.pth", map_location=device)
model.load_state_dict(model_state_dict)

using cached model. /content/drive/MyDrive/종합설계/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


<All keys matched successfully>

# 4. test_set 예측 라벨 산출
- 이제 3870개의 test 문장을 모델에 투입해서, 예측된 라벨링을 얻을 거예요!
- 결과 라벨은 y_hat에 저장할게요~

In [16]:
def predict(predict_sentence):
  # 1. data set 구성 (문장, 라벨)
  data = [predict_sentence, '0']
  dataset_another = [data]

  # 2. data를 bert의 입력에 맞게 변환하기
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=0)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)

      # 모델 forward 연산 진행
      out = model(token_ids, valid_length, segment_ids)
      
      # torch out -> numpy 형식으로 변환
      logits = out[0].detach().cpu().numpy()

      return np.argmax(logits)

In [17]:
y_hat = list()

for rows in test_set:
  labels = predict(rows[0])
  y_hat.append(labels)

In [18]:
y_hat[:3]

# 이게 저희 모델로 예측한 라벨 값들 입니다!

[1, 0, 2]

In [20]:
y = list()

for rows in test_set:
  y.append(rows[1])

# test_set은 [문장, 실제 라벨값]의 구조라서 실제 라벨값만 따로 y에 저장했어요 

In [21]:
y[:3]

# 요건 실제 라벨값 입니다

[1, 0, 2]

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

cm = confusion_matrix(y, y_hat)
print("===Confusion Matrix===")
print(cm)

# y, y_hat으로 confusion matrix을 산출한 결과입니다
# row는 실제 라벨
# col은 예측 라벨
# 즉, 0행은 실제 일반 문장 1999개에서 (1912 + 54 + 33) 1912개는 올바르게 평가, 54개는 폭언으로 잘못 산출, 33개는 성희롱으로 잘못 산출 했다는 것을 의미합니다
# 우리의 목표는 54와 33으로 잘못 판별된 문장들을 모델 투입 전에 사전에서 미리 거를 수 있도록, 사전, 문장 각색을 진행하는 것입니다!

===Confusion Matrix===
[[1912   54   33]
 [  62  906   43]
 [  55   42  763]]


# 5. 사전 각색 or 문장 각색
- 이제 본격적인 알고리즘 작성이 필요합니다
- 대략적인 순서는 제가 여기에 적어둘게여
  - y 와 y_hat을 비교하면서, y==0 인데, y_hat != 0이 아닌 문장의 인덱스를 구합니다!
  - 이제, 구한 인덱스를 바탕으로 test_set에서 문장을 찾아서, 형태소 분해 후 저장을 합니다
  - 저장된 형태소 리스트를 바탕으로, 각각 사전의 어떤 키워드와 매칭되는지 확인합니다!
  - 마지막으로, 사전 각색 혹은 문장 각색 두 방법 중 하나를 선택해서 진행하시면 됩니다!


In [27]:
# 1. y 와 y_hat을 비교하면서, y==0 인데, y_hat != 0이 아닌 문장의 인덱스를 구합니다! (== 실제 일반 문장인데, 폭언, 성희롱으로 잘못 예측된 문장의 인덱스)

moral_abuse_idx = list() # 일반인데 폭언으로 잘못 판별된 문장의 인덱스 리스트
moral_sexual_idx = list() # 일반인데 성희롱으로 잘못 판별된 문장의 인덱스 리스트

for idx in range(len(y)):
  if y[idx] == 0 and y_hat[idx] == 1:
    moral_abuse_idx.append(idx)
  elif y[idx] == 0 and y_hat[idx] == 2:
    moral_sexual_idx.append(idx)


# 구한 인덱스의 길이가 confusion matrix의 54, 33으로 나오겠죠?
len(moral_abuse_idx), len(moral_sexual_idx)

(54, 33)

In [111]:
# 2. 찾은 인덱스를 바탕으로 test_set에서 문장을 찾고, 형탠소 분해 후 저장해요
from kiwipiepy import Kiwi
kiwi = Kiwi()

moral_abuse_morphs = list()    # 일반인데 폭언으로 잘못 판별된 문장의 형태소 리스트
moral_sexual_morphs = list()   # 일반인데 성희롱으로 잘못 판별된 문장의 형태소 리스트

for idx in moral_abuse_idx:
  morphs = kiwi.tokenize(test_set[idx][0])
  temp = list()
  for morph in morphs:
    temp.append(morph.form)
  moral_abuse_morphs.append([idx,temp])

for idx in moral_sexual_idx:
  morphs = kiwi.tokenize(test_set[idx][0])
  temp = list()
  for morph in morphs:
    temp.append(morph.form)
  moral_sexual_morphs.append([idx,temp])

print(moral_abuse_morphs[0])
print(moral_sexual_morphs[0])

# 출력된 결과를 보면, 짱개, 성관계 등의 형태소가 포함되어 있어서 두 문장에 대한 표현은 사실 그냥 넘어가야 합니다 ㅜ
# 아래 문장들은 사실, 일반 문장 데이터 만들 때 제거되었어야 맞는데.. 흠.. 누군가 대충 넘어가서 이런 결과가.. ㅎ

[33, ['야', '짱개', '언어', '그냥', '모르', '어도', '되', '거든', '?']]
[25, ['성관계', '끝나', '었', '는데', '도', '계속', '이쁘', '다고', '하', '어', '주', '고', '이름', '이랑', '전화', '번호', '도', '알리', '어', '주', '네', '.']]


In [112]:
# 3. 우선, 폭언으로 잘못 판별된 문장들이 사전의 어느 부분이랑 매칭되는지 확인해볼까요?
# 이때, 사전은 폭언, 성희롱 사전 모두 사용하셔야 됩니다 --> 실제 시스템에서 두개다 매칭을 진행할거라서 ㅇㅇ

for rows in moral_abuse_morphs:
  abuse_matched = list()
  sexual_matched = list()

  for morph in rows[1]:
    if morph in abuse_list:
      abuse_matched.append(morph)
    if morph in sexual_list:
      sexual_matched.append(morph)

  rows.append(abuse_matched)
  rows.append(sexual_matched)

for rows in moral_abuse_morphs:
  print(rows)
  print()

# 폭언으로 잘못 분류된 문장 모두에 대해 (test_set 인덱스, 문장 형태소, 폭언 사전 매칭 형태소, 성희롱 사전 매칭 형태소)
# 보면 1396이랑,2752 인덱스는 어떤 사전이랑도 매칭되는 게 없잖아요. 바로 해당 문장들이 사전을 적용하게 되면, 매칭이 되는게 없어서 바로 일반 문장으로 판단되는 문장들입니다!
# 제가 말했었죠. 그 사전 적용하면 딱 2문장만 걸러진다고요. 그게 바로 이 두문장입니다!!
# 그러니까, 지금 저희가 하는 일이. 이 두문장 말고도 사전에 매칭되는 문장이 없게끔 문장, 사전을 각색하는 일입니다!

[33, ['야', '짱개', '언어', '그냥', '모르', '어도', '되', '거든', '?'], ['짱개', '어도'], ['어도']]

[104, ['걔', '네', '이름', '부터', '정', '떨어지', 'ᆷ', 'ㅋㅋ', '평생', '땅', '치', '고', '후회', '하', '었', '으면'], ['정', '떨어지', '땅', '치'], ['정', '으면']]

[135, ['안락사', '는', '죽음', '을', '존엄', '하', '게', '맞', '을', '수', '있', '게', '돕', '자는', '거', '이', 'ᆫ데', '.'], ['안락사', '죽음', '맞', '자는'], ['자는']]

[171, ['저번', '대선', '홍', '이', '문', '개털', '었', '는데', '?', '탄핵', '때문', '에', '지', 'ᆫ', '거', '이', '야'], ['홍'], ['는데']]

[246, ['너', '아이스크림', '먹', '고', '싶', '다며', '나', '가', '배', '민', '으로', '시키', '어', '주', 'ᆯ까', '?'], ['먹', '시키'], ['먹', '배', '시키', '주']]

[368, ['나', '그냥', '멍', '때리', '고', '있', '었', '어'], ['멍', '때리'], ['때리']]

[413, ['감방', '가', '어서', '도', '또', '사기치', '어', '먹', '을', '궁리', '하', '겠', '지', '점점', '더', '지능', '것', '으로', '크', 'ᆫ', '사기치력', '이', '고', '열심히', '연구', '하', '겠', '구만', '하', '는', '넘', '은', '또', '하', '지', '.'], ['감방', '사기치', '먹', '지능', '크', '넘'], ['어서', '먹', '크']]

[417, ['아니야', '장애', '있', '는', '사람', '들', '도', '결혼', '잘', '하', '어서', 

In [113]:
# 4. 사전 각색 방법

# 위에 문장 중에서, 171 인덱스 문장을 바탕으로 사전을 각색하는게 좋아보여요
# 폭언 사전에서는 '홍', 성희롱 사전에서는 '는데'만 매칭되었거든요
# 그럼 해당 키워드를 제거하기 전에, 기존에 '홍', '는데' 키워드만 가지고 있던 폭언, 성희롱 문장이 있을 수도 있어요
# 그런 문장이 있는지 확인해봐여

def isOnlyThatKeyword(keyword):
  res = list()

  for rows in test_set:
    if rows[1] == 1 or rows[1] == 2:  
      temp_all = list()
      temp_abuse = list()
      temp_sexual = list()
      morphs = kiwi.tokenize(rows[0])

      flag = False
      for morph in morphs:
        if morph.form == keyword:
          flag =True
      
      if flag == True:
        for morph in morphs:
          temp_all.append(morph.form)

          if morph.form in abuse_list:
            temp_abuse.append(morph.form)
          if morph.form in sexual_list:
            temp_sexual.append(morph.form)
        
      if len(temp_abuse) + len(temp_sexual) == 1:
        res.append([temp_all, temp_abuse, temp_sexual])
      

  return res


In [114]:
isOnlyThatKeyword('홍')

[]

In [115]:
isOnlyThatKeyword('는데') # 홍, 는데만으로 매칭되는 폭언, 성희롱 문장은 업네여!

[]

In [116]:
# 홍, 는데 모두 각 키워드만으로 매칭되는 실제 폭언, 성희롱 문장이 없다네요
# 즉, 사전에서 해당 두 키워드를 즉시 삭제해도, 무관하다는 뜻입니다
# 이렇게 경우의 수를 전부 계산해서 사전 csv 에서 키워드를 삭제하면 됩니다
# 단, 유의할 점은 이렇게 사전에서 키워드를 삭제하면, 다른 문장 검증시 갱신된 사전을 바탕으로 진행해야 합니다 --> 그냥 하면 미세한 확률로 에러가 날 수도! ㅎ

# 그래서 두번째 방법을 알려드립니다

print(test_set[171])

['저번대선 홍이 문 개털었는데? 탄핵때문에 진거야', 0]


In [117]:
# 5. 문장 각색

# 예를 들어, 위의 171번 문장에서 문제가 되는게, [홍, 는데] 두개니까
example = "저번대선 김이 문 이김, 탄핵때문에 진거야"
# 이런식으로 홍,는데 를 없애서 회피하듯이 문장을 다시 짜는 겁니다

In [118]:
# 해당 문장은 문제가 없는지 검증해볼까요

def validation(sentence):
  morphs = kiwi.tokenize(example)
  flag = False
  for morph in morphs:
    if morph.form in abuse_list or morph.form in sexual_list:
      print(morph.form)
      flag = True
  
  if flag == False:
    print("문장 각색 완료")
  else:
    print("위의 키워드가 문제가 됩니다. 문장을 다시 각색하세요!")

In [119]:
validation(example)

김
이기
위의 키워드가 문제가 됩니다. 문장을 다시 각색하세요!


In [120]:
# 각색 결과 또 문제가 생기네여 ㅋㅋㅋㅋㅋㅋㅋㅋ
# 그럼 이번에는 이렇게 바꿀게여
example = "저번대선 윤이 문 개텀, 탄핵때문에 진거야"
validation(example)

문장 각색 완료


In [121]:
# 이번에는 문제가 없네여!
# 그럼 해당 문장을 csv에서 바꿔주면 됩니다!

# ㅎㅎ.. 복잡하죠 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
# 사전 각색 vs 문장 각색 비교하면, 아무래도 문장 각색이 신경쓸게 덜합니다  
# 사전 각색이 귀찮은 이유는 사전에서 키워드를 제거하고 갱신된 사전을 다시 코랩으로 로드하고 다음 문장을 작업해야하기 때문입니다 (갱신 안하고 작업하면, 정확한 산출이 불가)

# 그래서 왠만하면 적절하게 문장을 각색하고 검증하는 방식을 이용하면 됩니다!
# 하지만, 제가 사용한 예시 문장처럼 '는데' 이런 말도안되는 키워드가 문제가 되면, 아무래도 사전 자체에서 삭제하는 편이 좋겠죠?
# --> 판단은 알아서!, 사전 각색이 귀찮으면, 전부 문장 각색으로 하셔도 됩니다~ --> 제가 봤을 때는 '는데'만 삭제하고 이후에는 그냥 문장 각색으로 빠르게 해도 될 것 같네요

# 위의 알고리즘에서 3번부터는 폭언으로 잘 못 판별한 문장으로만 진행 중입니다
# 그러니까 성희롱으로 잘 못 판별한 문장에서도 각색이 필요해요!
# 성희롱 코드는 아래에서 확인!

# 이렇게 30문장 정도만 추가로 거르면 좋을 것 같아요! 고생하세여! ㅎ  (죄송해요ㅜ 많으면 많을 수록 좋을 것 같아요..ㅎ)
# 아 각색 과정에서, 사전을 수정했으면, 어떤 키워드를 수정했는지
# 문장을 수정했으면 어떤 문장을 수정했는지 인덱스 정보를 기록해두세요!

In [122]:
# 부록. 성희롱으로 잘 못 판별한 문장

for rows in moral_sexual_morphs:
  abuse_matched = list()
  sexual_matched = list()

  for morph in rows[1]:
    if morph in abuse_list:
      abuse_matched.append(morph)
    if morph in sexual_list:
      sexual_matched.append(morph)

  rows.append(abuse_matched)
  rows.append(sexual_matched)

for rows in moral_sexual_morphs:
  print(rows)
  print()

# 성희롱으로 잘못 분류된 문장 모두에 대해 (test_set 인덱스, 문장 형태소, 폭언 사전 매칭 형태소, 성희롱 사전 매칭 형태소)
# 제가 봤을 때는 1102, 2078, 2267 요런 문장들이 각색하기 편해 보이네여

[25, ['성관계', '끝나', '었', '는데', '도', '계속', '이쁘', '다고', '하', '어', '주', '고', '이름', '이랑', '전화', '번호', '도', '알리', '어', '주', '네', '.'], ['알리'], ['성관계', '는데', '주', '주']]

[142, ['애견', '카페', '가', '었', '는데', '나', '의', '옆', '에서', '강아지', '들', '짝짓기', '하', 'ᆷ', '..'], ['들'], ['는데', '옆', '들', '짝짓기']]

[155, ['새', '거', '도', '아니', 'ᆫ', '스타킹', '으로', '포장', '되', '어', '있', '음'], ['새'], ['스타킹']]

[159, ['러시아', '여자', '들', '이', '리얼', '예쁘', 'ᆷ'], ['여자', '들', '예쁘'], ['여자', '들', '예쁘']]

[285, ['잘', '찍', '고', '오', '었', '지', '~', '근데', '가슴', '이', '작', '어서', '곤란', '하', '었', '음'], ['잘', '찍', '가슴', '작'], ['잘', '찍', '가슴', '작', '어서']]

[301, ['무슨', '대단', '하', 'ᆫ', '집', '아들', '이', 'ᆫ', '줄', '알', '었', '더니', '남자', '개털', '이', '구만'], ['아들', '줄'], ['무슨', '줄', '알', '남자']]

[569, ['넘', '귀엽', '지', '않', '냐', '??', '저', '정도', '이', '면', '강아지', '탈', '을', '쓰', 'ᆫ', '사람', '아니', 'ᆯ까', 'ㅋㅋ'], ['넘', '탈', '쓰', '사람'], ['냐', '탈', '쓰']]

[1091, ['ㄹㅇ', '거기', '에', '돈', '쓰', '는', '아재', '가', '얼마나', '많', '은데'], ['거기', '쓰', '아재', '많'], ['거기', '돈'